In [2]:
import pyterrier as pt



In [3]:
if not pt.started():
  pt.init()

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [4]:
def msmarco_generate():
    dataset = pt.get_dataset("trec-deep-learning-passages")
    with pt.io.autoopen('collections/msmarco-passage/collection.tsv', 'rt') as corpusfile:
        for l in corpusfile:
            docno, passage = l.split("\t")
            yield {'docno' : docno, 'text' : passage}

In [5]:
dataset = pt.get_dataset("trec-deep-learning-passages")

In [6]:
# iter_indexer = pt.IterDictIndexer("./passage_index")
# indexref = iter_indexer.index(msmarco_generate(), meta={'docno' : 20, 'text': 4096})
index = pt.FilesIndexer("./passage_index",meta={'docno' : 20, 'text': 4096})

In [7]:
tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
bm25b = pt.BatchRetrieve(index, controls={"wmodel":"BM25"})
bm25_qe = pt.BatchRetrieve(index, wmodel="BM25", controls={"qe":"on", "qemodel" : "Bo1"})
# pl2 = pt.BatchRetrieve(index, wmodel="PL2")

18:07:15.498 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
18:07:17.555 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
18:07:19.939 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
18:07:22.066 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.


In [19]:
result = pt.Experiment([bm25], 
                       dataset.get_topics("test-2019"), 
                       dataset.get_qrels("test-2019"),
                       eval_metrics=["recip_rank", "ndcg_cut_10","map", "mrt"])
print(result)

       name  recip_rank  ndcg_cut_10       map         mrt
0  BR(BM25)    0.795028      0.47954  0.370004  259.930386


In [20]:
result_TF = pt.Experiment([tf_idf], 
                       dataset.get_topics("test-2019"), 
                       dataset.get_qrels("test-2019"),
                       eval_metrics=["recip_rank", "ndcg_cut_10","map", "mrt"])
print(result_TF)

         name  recip_rank  ndcg_cut_10       map         mrt
0  BR(TF_IDF)    0.795028      0.47831  0.369486  140.222573


In [21]:
result_qe= pt.Experiment([bm25_qe], 
                       dataset.get_topics("test-2019"), 
                       dataset.get_qrels("test-2019"),
                       eval_metrics=["recip_rank", "ndcg_cut_10","map","mrt"])
print(result_qe)

       name  recip_rank  ndcg_cut_10       map        mrt
0  BR(BM25)    0.788124      0.50862  0.399897  277.65259


In [22]:
result_tf = pt.Experiment([tf_idf], 
                       dataset.get_topics("test-2019"), 
                       dataset.get_qrels("test-2019"),
                       eval_metrics=["recip_rank", "ndcg_cut_10","map","mrt"])
print(result_tf)

         name  recip_rank  ndcg_cut_10       map         mrt
0  BR(TF_IDF)    0.795028      0.47831  0.369486  108.719731


In [23]:
result_per_query = pt.Experiment([bm25], 
                                 dataset.get_topics("test-2019"),
                                 dataset.get_qrels("test-2019"), 
                                 eval_metrics=["recip_rank", "ndcg_cut_10","map","mrt"],
                                 perquery=True)

/usr/local/lib/python3.8/dist-packages/pyterrier/pipelines.py:108: UserWarning: 157 topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.
  warn(f'{backfill_count} topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.')


In [43]:
result_per_query = result_per_query[result_per_query["value"].notnull()]
result_per_query[result_per_query["measure"]=='map'].sort_values(by="value")

# result_per_query[result_per_query["measure"]=="ndcg_cut_10"].sort_values(by="value")

,name,qid,measure,value
96,BR(BM25),1121709,map,0.000239
6,BR(BM25),1063750,map,0.003193
93,BR(BM25),443396,map,0.005157
12,BR(BM25),489204,map,0.055242
39,BR(BM25),962179,map,0.058701
24,BR(BM25),527433,map,0.064043
78,BR(BM25),1106007,map,0.094742
27,BR(BM25),1037798,map,0.109533
48,BR(BM25),451602,map,0.122208
66,BR(BM25),833860,map,0.139674


Query Expansion

In [33]:
Bo1 = pt.rewrite.Bo1QueryExpansion(index)
KL = pt.rewrite.KLQueryExpansion(index)
# RM3 = pt.rewrite.RM3(index)



15:59:55.615 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
15:59:57.962 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.


In [34]:
pt.Experiment(
    [
            bm25, 
            bm25 >> Bo1 >> bm25, 
            bm25 >> KL >> bm25, 
            
    ],
    dataset.get_topics('test-2019'),
    dataset.get_qrels("test-2019"),
    eval_metrics=["recip_rank", "ndcg_cut_10","map"],
    names=["BM25", "+Bo1", "+KL"]
    )

,name,recip_rank,ndcg_cut_10,map
0,BM25,0.795028,0.479540,0.370004
1,+Bo1,0.788124,0.508620,0.399897
2,+KL,0.785023,0.505715,0.397658


In [35]:
result_per_query_Bo = pt.Experiment([bm25 >> Bo1 >> bm25], 
                                 dataset.get_topics("test-2019"),
                                 dataset.get_qrels("test-2019"), 
                                 eval_metrics=["recip_rank", "ndcg_cut_10","map"],
                                 perquery=True)

/usr/local/lib/python3.8/dist-packages/pyterrier/pipelines.py:108: UserWarning: 157 topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.
  warn(f'{backfill_count} topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.')


In [36]:
result_per_query_KL = pt.Experiment([bm25 >> KL >> bm25], 
                                 dataset.get_topics("test-2019"),
                                 dataset.get_qrels("test-2019"), 
                                 eval_metrics=["recip_rank", "ndcg_cut_10","map"],
                                 perquery=True)

/usr/local/lib/python3.8/dist-packages/pyterrier/pipelines.py:108: UserWarning: 157 topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.
  warn(f'{backfill_count} topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.')


In [45]:
result_per_query_Bo = result_per_query_Bo[result_per_query_Bo["value"].notnull()]
# result_per_query_Bo[result_per_query_Bo["measure"]=="ndcg_cut_10"].sort_values(by="value")
result_per_query_Bo[result_per_query_Bo["measure"]=="map"].sort_values(by="value")

,name,qid,measure,value
39,"Compose(Compose(BR(BM25), <pyterrier.rewrite.B...",1121709,map,0.000158
6,"Compose(Compose(BR(BM25), <pyterrier.rewrite.B...",1063750,map,0.002752
90,"Compose(Compose(BR(BM25), <pyterrier.rewrite.B...",443396,map,0.004997
12,"Compose(Compose(BR(BM25), <pyterrier.rewrite.B...",1106007,map,0.019114
105,"Compose(Compose(BR(BM25), <pyterrier.rewrite.B...",527433,map,0.038237
126,"Compose(Compose(BR(BM25), <pyterrier.rewrite.B...",962179,map,0.052588
99,"Compose(Compose(BR(BM25), <pyterrier.rewrite.B...",489204,map,0.062827
93,"Compose(Compose(BR(BM25), <pyterrier.rewrite.B...",451602,map,0.126669
15,"Compose(Compose(BR(BM25), <pyterrier.rewrite.B...",1110199,map,0.135263
0,"Compose(Compose(BR(BM25), <pyterrier.rewrite.B...",1037798,map,0.138890


In [44]:
result_per_query_KL = result_per_query_KL[result_per_query_KL["value"].notnull()]
result_per_query_KL[result_per_query_KL["measure"]=='map'].sort_values(by="value")

# result_per_query_KL[result_per_query_KL["measure"]=="ndcg_cut_10"].sort_values(by="value")

,name,qid,measure,value
39,"Compose(Compose(BR(BM25), <pyterrier.rewrite.K...",1121709,map,0.000159
6,"Compose(Compose(BR(BM25), <pyterrier.rewrite.K...",1063750,map,0.002744
90,"Compose(Compose(BR(BM25), <pyterrier.rewrite.K...",443396,map,0.004971
12,"Compose(Compose(BR(BM25), <pyterrier.rewrite.K...",1106007,map,0.018342
105,"Compose(Compose(BR(BM25), <pyterrier.rewrite.K...",527433,map,0.030683
126,"Compose(Compose(BR(BM25), <pyterrier.rewrite.K...",962179,map,0.052646
99,"Compose(Compose(BR(BM25), <pyterrier.rewrite.K...",489204,map,0.062751
93,"Compose(Compose(BR(BM25), <pyterrier.rewrite.K...",451602,map,0.126966
15,"Compose(Compose(BR(BM25), <pyterrier.rewrite.K...",1110199,map,0.135193
0,"Compose(Compose(BR(BM25), <pyterrier.rewrite.K...",1037798,map,0.142711
